In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path folder dataset di Google Drive
train_dir = '/content/drive/Shared drives/ML - Skinalyze/dataset/train'
val_dir = '/content/drive/Shared drives/ML - Skinalyze/dataset/test'

In [ ]:
# Data Augmentation dan Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load Data dari folder Google Drive
train_data = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 1964 images belonging to 7 classes.
Found 267 images belonging to 7 classes.


In [ ]:
from tensorflow.keras import Input

# Model CNN
model = models.Sequential([
    Input(shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

In [ ]:
# Optimizer dengan LR dan Momentum
optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.03)

In [ ]:
# Compile model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

In [ ]:
# Callbacks: Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_auc',
    patience=10,
    restore_best_weights=True
)

In [ ]:
# Training Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=100,
    callbacks=[early_stopping]
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 1628s 41s/step - accuracy: 0.2006 - auc: 0.5845 - loss: 1.9145 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1461 - val_auc: 0.5301 - val_loss: 1.9596 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 197s 6s/step - accuracy: 0.2380 - auc: 0.6351 - loss: 1.8766 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1461 - val_auc: 0.5891 - val_loss: 1.9353 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 186s 6s/step - accuracy: 0.2697 - auc: 0.6636 - loss: 1.8397 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1835 - val_auc: 0.5616 - val_loss: 1.9735 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 200s 6s/step - accuracy: 0.2990 - auc: 0.6782 - loss: 1.8062 - precision: 0.0938 - recall: 4.8270e-05 - val_accuracy: 0.2659 - val_auc: 0.6680 - val_loss: 1.8191 - val_precision: 0.0000e+00 - val_r

In [ ]:
# Evaluasi Model
print("Evaluasi Model:")
evaluation = model.evaluate(val_data)
for name, value in zip(model.metrics_names, evaluation):
    print(f"{name}: {value}")

Evaluasi Model:
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8189 - auc: 0.9787 - loss: 0.5401 - precision: 0.8816 - recall: 0.7228
loss: 0.5266112685203552
compile_metrics: 0.8277153372764587


In [ ]:
# Save Model
model.save('/content/drive/My Drive/Skinalyze.h5')
print("Model berhasil disimpan di Google Drive.")

Model berhasil disimpan di Google Drive.
